My attempt at reproducing Dave's `snodka` function. Dave's function will be wrong when the atmosphere is changing faster than a scan.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits

from dysh.fits.gbtfitsload import GBTFITSLoad

In [ ]:
def parse_query(query, op="and"):
    """ Takes a dictionary with keys as SDFITS columns
    and whose values are a tuple with the condition and 
    the required value and concatenates them using the
    `op` operator.
    """
    q = ""
    for i,(k,v) in enumerate(query.items()):
        q += f"{k} {v[0]} {v[1]} "
        if i < len(query) - 1:
            q += f"{op} "
    return q


def select_rows(df, query, op="and"):
    """ Returns a list with the rows of a `pandas.DataFrame`
    that fulfill the query.
    See `parse_query` for details on how to define the query.
    """

    q = parse_query(query, op=op)
    return list(df.query(q).index)

In [ ]:
path = "/home/dysh/example_data/subbeamnod-Ka/"
sdf_file = f"{path}/data/TRCO_230413_Ka.raw.vegas/TRCO_230413_Ka.raw.vegas.A.fits"

In [ ]:
sdf = GBTFITSLoad(sdf_file)

In [ ]:
# Define what we are calibrating.
scan = 43
ifnum = 0
fdnum = 1 # Remember that Ka is special because its beams are switched! It is now fixed, yay!
plnum = 0
bintable = 0
w = "tsys"
docal = True

# Select data.
q = {"SCAN": ("in", f"[{scan}]"), 
     "IFNUM": ("in", f"[{ifnum}]"), 
     "FDNUM": ("in", f"[{fdnum}]"), 
     "PLNUM": ("==", plnum), 
     "SUBREF_STATE": ("==", -1)}
sig_rows = select_rows(sdf._ptable[bintable], q)
q = {"SCAN": ("in", f"[{scan}]"), 
     "IFNUM": ("in", f"[{ifnum}]"), 
     "FDNUM": ("in", f"[{fdnum}]"), 
     "PLNUM": ("==", plnum), 
     "SUBREF_STATE": ("==", 1)}
ref_rows = select_rows(sdf._ptable[bintable], q)

# Average.
def time_average(table):
    wt = np.empty(len(table), dtype='d')
    wt[:] = table["EXPOSURE"].astype('d')*abs(table["CDELT1"]).astype('d')/table["TSYS"].astype('d')
    return np.average(table["DATA"], axis=0, weights=wt)

ref_avg = time_average(sdf._hdu[bintable+1].data[ref_rows])
sig_avg = time_average(sdf._hdu[bintable+1].data[sig_rows])

# Get TSYS.
fulltp = sdf.gettp(scan,sig=None,cal=None,
                    bintable=bintable,fdnum=fdnum,
                    plnum=plnum,ifnum=ifnum,
                    weight=w,calibrate=docal).timeaverage(weights=w)

# Good old calibration.
cal = (sig_avg - ref_avg)/ref_avg * fulltp.meta['TSYS']

# Now we should make a new SDFITS row with the calibrated data 
# and the corresponding updated data and metadata.

In [ ]:
# Open the result from GBTIDL.
gbtidl_file = f"{path}/outputs/snodka_scan_43_fdnum_1_plnum_0.fits"
hdu_ = fits.open(gbtidl_file)
gbtidl_sbn = hdu_[1].data["DATA"][0]

In [ ]:
# Compare.
diff = cal - gbtidl_sbn
print(f"Mean diff: {np.nanmean(diff)}")
print(f"Median diff: {np.nanmedian(diff)}")

# Allways look at it.
plt.figure()
plt.subplot(211)
plt.plot(cal, label="dysh")
plt.plot(gbtidl_sbn, alpha=0.5, label="GBTIDL")
plt.legend()
plt.ylabel("Antenna temperature (K)")
plt.xlabel("Channel number")
plt.subplot(212)
plt.plot(cal - gbtidl_sbn, c='k')
plt.ylabel("Difference (K)")
plt.xlabel("Channel number");